In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input, Layer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.applications import VGG16

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# FEATURE MAP

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input, Layer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.applications import VGG16

# Define input size for VGG16
input_size = (224, 224, 3)
n_features = 16

class QuantumInspiredFeatureMap(Layer):
    def __init__(self, output_dim=16, **kwargs):
        super(QuantumInspiredFeatureMap, self).__init__(**kwargs)
        self.output_dim = output_dim

    def build(self, input_shape):
        # Trainable transformation weights
        self.kernel = self.add_weight(
            name='quantum_kernel',
            shape=(input_shape[-1], self.output_dim),
            initializer='glorot_uniform',
            trainable=True
        )

        # Bias term
        self.bias = self.add_weight(
            name='quantum_bias',
            shape=(self.output_dim,),
            initializer='zeros',
            trainable=True
        )

        super().build(input_shape)

    def call(self, inputs):
        # Nonlinear transformation inspired by quantum feature mapping
        # Uses trainable weights to create complex feature interactions

        # Linear transformation with trainable kernel
        transformed = tf.matmul(inputs, self.kernel)

        # Add bias
        transformed = tf.nn.bias_add(transformed, self.bias)

        # Nonlinear activation with sine (mimicking quantum phase)
        transformed = tf.math.sin(transformed)

        # Amplitude normalization
        transformed = tf.math.l2_normalize(transformed, axis=-1)

        return transformed

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

def create_hybrid_model(input_shape, num_classes):
    """
    Create hybrid neural network with quantum-inspired feature transformation
    """
    # Base VGG16 model
    base_model = VGG16(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    base_model.trainable = False  # Freeze base model layers

    # Input layer
    inputs = Input(shape=input_shape)

    # Feature extraction with VGG16
    x = base_model(inputs)

    # Flatten features
    x = Flatten()(x)

    # Dropout for regularization
    x = Dropout(0.5)(x)

    # Quantum-inspired feature transformation
    x = QuantumInspiredFeatureMap(output_dim=n_features)(x)

    # Additional dense layers
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)

    # Output layer
    outputs = Dense(num_classes, activation='softmax')(x)

    # Create and compile model
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Image data preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Data generators
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/PROJECTS/DATASETS/Intel Image Classification/seg_train/seg_train',
    target_size=input_size[:2],
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    '/content/drive/MyDrive/PROJECTS/DATASETS/Intel Image Classification/seg_test/seg_test',
    target_size=input_size[:2],
    batch_size=32,
    class_mode='categorical'
)

# Learning rate scheduler
def lr_scheduler(epoch, lr):
    decay_rate = 0.9
    decay_step = 5
    if epoch % decay_step == 0 and epoch:
        return lr * decay_rate
    return lr

lr_callback = LearningRateScheduler(lr_scheduler)

# Create and train the model
model = create_hybrid_model(input_size, train_generator.num_classes)

# Model summary
model.summary()

# Training
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[lr_callback]
)

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ vgg16 (Functional)                   │ (None, 7, 7, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ quantum_inspired_feature_map         │ (None, 16)                  │         401,424 │
│ (QuantumInspiredFeatureMap)          │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │           2,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │             774 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15,119,062 (57.67 MB)

 Trainable params: 404,374 (1.54 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 66/439 ━━━━━━━━━━━━━━━━━━━━ 2:16:53 22s/step - accuracy: 0.2286 - loss: 1.7689

KeyboardInterrupt: 

# CLASSICAL

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

# Directly specifying the input size
input_size = (224, 224)

# ImageDataGenerator for training data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# ImageDataGenerator for validation data
val_datagen = ImageDataGenerator(rescale=1./255)

# Load training data from directory
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/PROJECTS/DATASETS/Intel Image Classification/seg_train/seg_train',
    target_size=input_size,
    batch_size=32,
    class_mode='categorical'
)

# Load validation data from directory
val_generator = val_datagen.flow_from_directory(
    '/content/drive/MyDrive/PROJECTS/DATASETS/Intel Image Classification/seg_test/seg_test',
    target_size=input_size,
    batch_size=32,
    class_mode='categorical'
)

# Load the pre-trained VGG16 model and freeze the layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze VGG16 layers

# Add custom layers on top of VGG16
x = base_model.output
x = Conv2D(16, (3, 3), activation='relu')(x)  # CNN layer with 16 filters
x = GlobalAveragePooling2D()(x)  # Global average pooling layer
x = Flatten()(x)  # Flatten the output
x = Dense(256, activation='relu')(x)  # Fully connected layer
output = Dense(train_generator.num_classes, activation='softmax')(x)  # Output layer

# Create the final model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=10, validation_data=val_generator)


Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


162/439 ━━━━━━━━━━━━━━━━━━━━ 1:38:01 21s/step - accuracy: 0.5467 - loss: 1.0775

# WITH CIRCUIT CODE

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Layer


# Quantum device setup
n_qubits = 16 # Number of qubits
dev = qml.device("default.qubit", wires=n_qubits)

# Define the quantum circuit
@qml.qnode(dev)
def quantum_circuit(inputs):
    # Amplitude embedding
    qml.AmplitudeEmbedding(inputs, wires=range(n_qubits), normalize=True)

    # Quantum operations
    qml.RY(-0.009 * np.pi, wires=0)
    qml.RX(0.25 * np.pi, wires=0)
    qml.RZ(0.25 * np.pi, wires=0)
    qml.CNOT(wires=[0, 1])
    qml.RX(0.25 * np.pi, wires=1)
    qml.RZ(0.25 * np.pi, wires=1)
    qml.RY(0.002 * np.pi, wires=1)
    qml.CNOT(wires=[1, 2])
    qml.RX(0.25 * np.pi, wires=2)
    qml.RZ(0.25 * np.pi, wires=2)
    qml.CNOT(wires=[2, 3])
    qml.RX(0.25 * np.pi, wires=3)
    qml.RZ(0.25 * np.pi, wires=3)

    # Return expectations
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

class QuantumLayer(Layer):
    def __init__(self, output_dim, **kwargs):
        """
        A custom Keras layer that integrates a quantum circuit with trainable weights.
        """
        self.output_dim = output_dim
        super(QuantumLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Trainable transformation weights
        self.kernel = self.add_weight(
            name='quantum_kernel',
            shape=(input_shape[-1], self.output_dim),
            initializer='glorot_uniform',
            trainable=True
        )
        self.bias = self.add_weight(
            name='quantum_bias',
            shape=(self.output_dim,),
            initializer='zeros',
            trainable=True
        )
        super(QuantumLayer, self).build(input_shape)

    def call(self, inputs):
        # Linear transformation before quantum circuit
        transformed_inputs = tf.matmul(inputs, self.kernel) + self.bias

        # Ensure inputs are in the correct shape for the quantum circuit
        quantum_outputs = tf.map_fn(lambda x: quantum_circuit(x), transformed_inputs)

        return quantum_outputs

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


# Define the hybrid model
def create_hybrid_model(input_shape, num_classes):
    """
    Create a hybrid classical-quantum neural network model.
    """
    # Base VGG16 model
    base_model = VGG16(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    base_model.trainable = False  # Freeze base model layers

    # Input layer
    inputs = Input(shape=input_shape)

    # Feature extraction with VGG16
    x = base_model(inputs)
    x = Flatten()(x)
    x = Dropout(0.5)(x)

    # Quantum-inspired transformation (Quantum Layer)
    x = Dense(2**n_qubits, activation='relu')(x)  # Match input size for quantum circuit
    x = QuantumLayer(output_dim=n_qubits)(x)

    # Additional dense layers
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    # Create and compile model
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


# Image data preprocessing
input_size = (224, 224, 3)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Data generators
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/PROJECTS/DATASETS/Intel Image Classification/seg_train/seg_train',
    target_size=input_size[:2],
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    '/content/drive/MyDrive/PROJECTS/DATASETS/Intel Image Classification/seg_test/seg_test',
    target_size=input_size[:2],
    batch_size=32,
    class_mode='categorical'
)

# Learning rate scheduler
def lr_scheduler(epoch, lr):
    decay_rate = 0.9
    decay_step = 5
    if epoch % decay_step == 0 and epoch:
        return lr * decay_rate
    return lr

lr_callback = LearningRateScheduler(lr_scheduler)

# Create and train the model
model = create_hybrid_model(input_size, train_generator.num_classes)

# Model summary
model.summary()

# Number of images to use
train_images = 100
val_images = 10

# Number of batches needed (based on batch size)
batch_size = 32

# Steps per epoch and validation steps
steps_per_epoch = train_images // batch_size
validation_steps = val_images // batch_size

# Train the model with a limited number of images
history = model.fit(
    train_generator,
    epochs=20,
    steps_per_epoch=steps_per_epoch,  # Limit the number of batches per epoch
    validation_data=val_generator,
    validation_steps=validation_steps,  # Limit the number of validation batches
    callbacks=[lr_callback]
)



# CIRCUIT USE

In [ ]:
import pennylane as qml
import numpy as np

# Define the device with 4 wires
dev = qml.device("default.qubit", wires=4)

# Define the quantum circuit
@qml.qnode(dev)
def circuit():
    qml.RY(-0.009 * np.pi, wires=0)
    qml.RX(0.25 * np.pi, wires=0)
    qml.RZ(0.25 * np.pi, wires=0)
    qml.CNOT(wires=[0, 1])
    qml.RX(0.25 * np.pi, wires=1)
    qml.RZ(0.25 * np.pi, wires=1)
    qml.RY(0.002 * np.pi, wires=1)
    qml.CNOT(wires=[1, 2])
    qml.RX(0.25 * np.pi, wires=2)
    qml.RZ(0.25 * np.pi, wires=2)
    qml.CNOT(wires=[2, 3])
    qml.RX(0.25 * np.pi, wires=3)
    qml.RZ(0.25 * np.pi, wires=3)
    return qml.expval(qml.PauliZ(0))

# Draw the circuit
drawer = qml.draw(circuit)
print(drawer())


0: ──RY(-0.03)──RX(0.79)──RZ(0.79)─╭●────────────────────────────────────────────────────────
1: ────────────────────────────────╰X──RX(0.79)──RZ(0.79)──RY(0.01)─╭●───────────────────────
2: ─────────────────────────────────────────────────────────────────╰X──RX(0.79)──RZ(0.79)─╭●
3: ────────────────────────────────────────────────────────────────────────────────────────╰X

──────────────────────┤  <Z>
──────────────────────┤     
──────────────────────┤     
───RX(0.79)──RZ(0.79)─┤     


In [ ]:
!pip install pennylane

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.0/930.0 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.2 MB/s eta 0:00:00
